## Understanding LSTM in Tensorflow(MNIST dataset)

参考：https://jasdeep06.github.io/posts/Understanding-LSTM-in-Tensorflow-MNIST/?spm=5176.100239.blogcont202939.11.snhVUr

通过学习大牛的这边文章，对lstm的理解更深入一步了。
长短期记忆（LSTM）是目前最常用的递归神经网络类型。文章的目的是让读者熟悉张量中基本LSTM网络的实现细节
为了实现这一目标，采用了MINIST作为我们的数据集，为什么要有MINIST数据集呢？

MNIST数据集由手写数字及其相应标签的图像组成。我们可以借助TensorFlow的内置功能下载和读取数据：

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/data/',one_hot=True)

数据分为三部分：
- 训练数据（mnist.train） - 55000个图像的训练数据
- 测试数据 （mnist.test）- 10000个图像的测试数据
- 验证数据（mnist.validation） -5000个图像的验证数据

数据形状：
训练数据包括55000个28x28像素的图像，这些（28x28）像素值以单个维度向量的形式被拉平，所有这样的55000个像素向量
的集合被存储为numpy阵列的形式（55000，784），并称为mnist.train.images。同时与这55000个对应的是其标签数据
一共10个类别标签，对应（0,1,2,3,4,5,6,7,8,9），标签以one_hot编码形式表示，因此被存储为（55000,10）称为mnist.labels

为什么用MNIST：
- LSTM通常用于复杂序列相关问题，NLP领域的语言建模，词嵌入，编码器等MNIST给了我们解决这类问题的机会，这里的输入数据
只是一组像素值，我们可以轻松地格式化这些值，并集中应用到问题的细节上。


### A RNN

!["RNN"](./pics/RNN1.png)

1、 Xt是指时间步长t的输入
2、 St是时间步长t处的隐藏状态，可以被看做是网络的“内存”
3、 Ot是时间步长t的输出
4、 U,V,W是所有时间步长共享的参数，该参数共享的意义在于，我们的模型在不同输入的时间步长可以执行相同的任务
我们通过展开RNN想要介绍的是，在每个时间步长中，网络可以看做是BP网络，同时要考虑到前一个时间步长的输出。


两个警告：
我们需要两个概念以便舒服的完成编码，
- 1、 LSTM cells的解释
- 2、在喂入tf RNNS之前格式化输入

1、LSTM cells的解释
基本的lstm声明：
tf.contrib.rnn.BasicLstmCell(num_units)
这里的num_units指的是LSTM cell中的单元个数，可以理解为BP中隐藏层的神经元个数,lstm的隐藏层神经元个数也是一样可以有多个unit啊
![](./pics/lstmcells.png)
每个num_units合起来就是一个标准的LSTM unit
![](./pics/stlstm.png)

### 2.输入格式化然后输入到RNNs中
tf中最简单的RNN形式是在static_rnn中定义：
tf.static_rnn(cell,inputs)
当然还有其他形式的方法，该inputs参数时为了接受形状张量列表[batch_size,input_size],该列表的长度是网络展开的时间步数，比如一个输入要通过n个时间单位那输入就是 n个[batch_size,input_size].

对于我们MNIST图像的情况，我们有大小为28x28的图像，他们可以被推断为具有28行28像素的图像，我们就通过n=28个时间步骤展开我们的网络，使得在每个时间步长我们可以输入一行像素（input_size）从而通过28个时间步长输入完整的图像。

如果我们提供batch_size图像的数量，每个时间步长将提供相应的batch_size图像行，下图应该可以解决所有疑问：
![](./pics/Visualizationofinputs.png)

static_rnn生成的输出是[batch_size,num_units]形状的张量列表,列表的长度是网络展开的时间步长数，即每个时间步长的一个输出张量，
在实现中我们只关注最后时间的输出作为预测。在lstm每一时刻的输入都会和上一时刻的输出做一个拼接，所以最后一个输入并不是和前面时刻输入毫无关联

In [ ]:
#-*-coding:utf-8 -*-
 
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data 
import os
#set GPU increase on demand
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config = config)

mnist = input_data.read_data_sets('.\\data\\', one_hot=True)
batch_size = tf.placeholder(tf.int32,[])

lr = 1e-3 #0.001
input_size = 28
timestep_size = 28

# nodes per hidden_layer
hidder_size = 256
#LSTM layers
layer_num = 2
#num of classes to predict
class_num = 10

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32,[None, class_num])
keep_prob = tf.placeholder(tf.float32)
 
#step1: shape =(batch_size,timestep_size,input_size)
input_data= tf.reshape(x, [-1,28,28])

def lstm_cell():
    return tf.contrib.rnn.BasicLSTMCell(num_units=hidder_size,state_is_tuple=True)
def dropout():
    cell = lstm_cell()
    return tf.contrib.rnn.DropoutWrapper(cell,output_keep_prob=keep_prob)
def multi_lstm():
    cells = [dropout() for _ in range(layer_num)]
    MultiRNN_cell = tf.contrib.rnn.MultiRNNCell(cells,state_is_tuple=True)
    return MultiRNN_cell

two_lstm_cell = multi_lstm()
init_state=two_lstm_cell.zero_state(batch_size, tf.float32)

#outputs,state=tf.nn.dynamic_rnn(two_lstm_cell,inputs=input_data,initial_state=init_state,time_major=False)
outputs = list()
state = init_state
with tf.variable_scope('RNN'):
    for timestep in range(timestep_size):
        if timestep > 0:
            tf.get_variable_scope().reuse_variables()
        # 这里的state保存了每一层 LSTM 的状态
        (cell_output, state) = two_lstm_cell(input_data[:, timestep, :], state)
        outputs.append(cell_output)
h_state = outputs[-1] # h_state: 只取了outputs的最后一状态 <tf.Tensor 'RNN/RNN/multi_rnn_cell/cell_1_27/dropout/mul:0' shape=(?, 256) dtype=float32>]
#lstm模型最终输出是最后一个时序的隐层维度，因此是256维，
# LSTM 部分的输出会是一个 [hidden_size] 的tensor，我们要分类的话，还需要接一个 softmax 层
out_W =tf.Variable(tf.truncated_normal(shape=[hidder_size,class_num],stddev=0.1),dtype=tf.float32)
out_B = tf.Variable(tf.constant(0.1,shape=[class_num,]),dtype=tf.float32)
#y_pre = tf.nn.softmax(tf.matmul(h_state,out_W) + out_B) #matmul:矩阵相乘
#cross_entropy = -tf.reduce_mean(y *tf.log(y_pre)) #reduce表示降维1/0、分别表示横向纵向降维，不传则横纵都降维，_fun表示降维的方式，求和或求均值等
y_pre = tf.matmul(h_state, out_W) + out_B  
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=y_pre))#使用tf提供的softmax交叉熵，准确率比上面的实现更高一点，logits必须是没有经过softmax层的，
train_op =tf.train.AdamOptimizer(lr).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_pre,1),tf.argmax(y,1))#argmax取vector中最大值的索引号
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))

sess.run(tf.global_variables_initializer())
tensorboard_dir = "./logs"
if not os.path.exists(tensorboard_dir):
    os.mkdir(tensorboard_dir)
writer = tf.summary.FileWriter(tensorboard_dir,sess.graph) 
tf.summary.scalar(name="cross_entropy", tensor = cross_entropy)
tf.summary.scalar(name="accuracy",tensor=accuracy)
merged_summary = tf.summary.merge_all()
 
save_per_step = 100
for i in range(2000):
    _batch_size = 128
    batch = mnist.train.next_batch(_batch_size)
    if (i+1) % save_per_step == 0:
        train_summary = sess.run(merged_summary,feed_dict={x:batch[0],
                                                      y:batch[1],
                                                      keep_prob:1.0,
                                                      batch_size:_batch_size
                                                      }
                                  )
        writer.add_summary(train_summary,(i+1))
    if (i+1)%200 == 0:
        train_accuracy = sess.run(accuracy,feed_dict={x:batch[0],
                                                      y:batch[1],
                                                      keep_prob:1.0,
                                                      batch_size:_batch_size
                                                      }
                                  )
        print("Iter {0},Step:{1} ,train_accuracy:{2}".format(mnist.train.epochs_completed,(i+1),train_accuracy))
    
    sess.run(train_op,feed_dict={x:batch[0],y:batch[1],keep_prob:0.5,batch_size:_batch_size})
print("test accuracy :{0}".format(sess.run(accuracy,feed_dict={x:mnist.test.images,
                                                               y:mnist.test.labels,
                                                               keep_prob:1.0,
                                                               batch_size:mnist.test.images.shape[0]
                                                               }
                                           )
                                  )
      )

附上一张张量图：
![](./pics/LSTM3.png)

上图可以理解为是上面程序单层lstm的张量处理过程，计算过程可以看下面的源码，看完源码后豁然开朗

In [ ]:
basicLSTMCell源码分析：
def call(self, inputs, state):
    """Long short-term memory cell (LSTM).

    Args:
      inputs: `2-D` tensor with shape `[batch_size, input_size]`.
      state: An `LSTMStateTuple` of state tensors, each shaped
        `[batch_size, self.state_size]`, if `state_is_tuple` has been set to
        `True`.  Otherwise, a `Tensor` shaped
        `[batch_size, 2 * self.state_size]`.

    Returns:
      A pair containing the new hidden state, and the new state (either a
        `LSTMStateTuple` or a concatenated state, depending on
        `state_is_tuple`).
    """
    sigmoid = math_ops.sigmoid
    one = constant_op.constant(1, dtype=dtypes.int32)
    # Parameters of gates are concatenated into one multiply for efficiency.
    if self._state_is_tuple:
      c, h = state #?,256
    else:
      c, h = array_ops.split(value=state, num_or_size_splits=2, axis=one)
    #将输入和上一个隐层按第二维度拼接，得到（？，step+hidden_size）， 
    gate_inputs = math_ops.matmul(
        array_ops.concat([inputs, h], 1), self._kernel) #input(?,28) h (?,256) kernel(284,1024) 
    gate_inputs = nn_ops.bias_add(gate_inputs, self._bias) #(?,1024)
    
    #将数据分割为四等分每个（？，256）
    # i = input_gate, j = new_input, f = forget_gate, o = output_gate
    i, j, f, o = array_ops.split( #(?,256)
        value=gate_inputs, num_or_size_splits=4, axis=one)

    forget_bias_tensor = constant_op.constant(self._forget_bias, dtype=f.dtype)
    # Note that using `add` and `multiply` instead of `+` and `*` gives a
    # performance improvement. So using those at the cost of readability.
    add = math_ops.add
    multiply = math_ops.multiply
    new_c = add(multiply(c, sigmoid(add(f, forget_bias_tensor))),#(?,256)
                multiply(sigmoid(i), self._activation(j)))
    new_h = multiply(self._activation(new_c), sigmoid(o)) #矩阵对应元素相乘(?,256)

    if self._state_is_tuple:
      new_state = LSTMStateTuple(new_c, new_h)#(?,256)
    else:
      new_state = array_ops.concat([new_c, new_h], 1)
    return new_h, new_state
